In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, classification_report
import matplotlib.pyplot as plt
import math

import scipy.stats as ss

In [2]:
dfD = pd.read_csv("../../SAheart.data")

In [3]:
display(dfD)

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
1,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
4,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
5,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1
...,...,...,...,...,...,...,...,...,...,...
459,214,0.40,5.98,31.72,Absent,64,28.45,0.00,58,0
460,182,4.20,4.41,32.10,Absent,52,28.61,18.72,52,1
461,108,3.00,1.59,15.23,Absent,40,20.09,26.64,55,0
462,118,5.40,11.61,30.79,Absent,64,27.35,23.97,40,0


In [4]:
le = preprocessing.LabelEncoder()
dfD["famhist"] = le.fit_transform(dfD["famhist"])

display(dfD)

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
1,160,12.00,5.73,23.11,1,49,25.30,97.20,52,1
2,144,0.01,4.41,28.61,0,55,28.87,2.06,63,1
3,118,0.08,3.48,32.28,1,52,29.14,3.81,46,0
4,170,7.50,6.41,38.03,1,51,31.99,24.26,58,1
5,134,13.60,3.50,27.78,1,60,25.99,57.34,49,1
...,...,...,...,...,...,...,...,...,...,...
459,214,0.40,5.98,31.72,0,64,28.45,0.00,58,0
460,182,4.20,4.41,32.10,0,52,28.61,18.72,52,1
461,108,3.00,1.59,15.23,0,40,20.09,26.64,55,0
462,118,5.40,11.61,30.79,0,64,27.35,23.97,40,0


In [5]:
scaler = preprocessing.MinMaxScaler()
X = dfD.drop(["chd"], axis = 1)

# Nao melhora em nada Scalar
#X = scaler.fit_transform(X)

y = dfD["chd"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(369, 9) (93, 9) (369,) (93,)


## SAHeat Gaussian

In [9]:
gNB = GaussianNB()

gNB.fit(X_train, y_train)
score = gNB.score(X_train, y_train)
ypred = gNB.predict(X_test)
cm = confusion_matrix(y_test, gNB.predict(X_test))

print(cm)

print(classification_report(y_test, ypred))

[[47 16]
 [11 19]]
              precision    recall  f1-score   support

           0       0.81      0.75      0.78        63
           1       0.54      0.63      0.58        30

    accuracy                           0.71        93
   macro avg       0.68      0.69      0.68        93
weighted avg       0.72      0.71      0.71        93



In [10]:
cv_results = cross_val_score(gNB, X_train, y_train,
                             cv = 10, #10 folds
                             scoring = "precision_macro",
                             verbose = 2
                            )
print("Precision:", cv_results.mean())

[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


## SAHeat LogisticRegression

In [11]:
logisticRegression = LogisticRegressionCV(cv=10, max_iter=1000)

logisticRegression.fit(X_train, y_train)
score = logisticRegression.score(X_train, y_train)
ypred = logisticRegression.predict(X_test)

cm = confusion_matrix(y_test, ypred)

# precision, recall, f1 metrics

print(cm)

print(classification_report(y_test, ypred))

[[48 15]
 [15 15]]
              precision    recall  f1-score   support

           0       0.76      0.76      0.76        63
           1       0.50      0.50      0.50        30

    accuracy                           0.68        93
   macro avg       0.63      0.63      0.63        93
weighted avg       0.68      0.68      0.68        93



In [12]:
logisticRegression = LogisticRegression(max_iter=1000)

logisticRegression.fit(X_train, y_train)
score = logisticRegression.score(X_train, y_train)
ypred = logisticRegression.predict(X_test)

cm = confusion_matrix(y_test, ypred)

# precision, recall, f1 metrics

print(cm)

print(classification_report(y_test, ypred))

[[51 12]
 [16 14]]
              precision    recall  f1-score   support

           0       0.76      0.81      0.78        63
           1       0.54      0.47      0.50        30

    accuracy                           0.70        93
   macro avg       0.65      0.64      0.64        93
weighted avg       0.69      0.70      0.69        93



/home/donghi/anaconda3/envs/ambDanielFiep/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
